# Store and retrieval

Make sure to have your huggingface api key provided to download the embedding model (see README.md).

## 1. Load and split the documents

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("./content/max-und-moritz.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size = 2500, chunk_overlap = 200)
docs = text_splitter.split_documents(documents)

## 2. Store documents into the vector database

In [2]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

# prepare the embedding model
embedding_function = SentenceTransformerEmbeddings(model_name = 'intfloat/multilingual-e5-large')

# load it into Chroma
db = Chroma(embedding_function = embedding_function, persist_directory = './chromadb/')
db.add_documents(docs)

['4e86dbc4-ca82-11ee-8cd9-f09e4ab13937',
 '4e86dbc5-ca82-11ee-86cc-f09e4ab13937',
 '4e86dbc6-ca82-11ee-9039-f09e4ab13937',
 '4e86dbc7-ca82-11ee-b6a3-f09e4ab13937',
 '4e86dbc8-ca82-11ee-b725-f09e4ab13937',
 '4e86dbc9-ca82-11ee-baf2-f09e4ab13937']

## 3. Query the documents

In [3]:
query = 'Wie heisst der Lehrer?'
#query = 'Who is the sensei?'
#query = 'Was machen die Hühner unmittelbar vor ihrem Tod?'
#query = 'Wie viele Hühner hatte Witwe Bolte?'
#query = 'Was machen Max und Moritz mit dem Schneider und wie heisst er?'

results = db.similarity_search_with_score(query)

for result in results:
    print(f'Score: {result[1]}')
    print(f'Text:\n{result[0].page_content}\n\n')

Score: 0.41745523551498115
Text:
– Nicht allein das A-B-C

Bringt den Menschen in die Höh’;

Nicht allein im Schreiben, Lesen

Übt sich ein vernünftig Wesen;

Nicht allein in Rechnungssachen

Soll der Mensch sich Mühe machen;

Sondern auch der Weisheit Lehren

Muß man mit Vergnügen hören. –

Daß dies mit Verstand geschah,

War Herr Lehrer Lämpel da. –

– Max und Moritz, diese beiden,

Mochten ihn darum nicht leiden;

Denn wer böse Streiche macht,

Gibt nicht auf den Lehrer acht. –

Nun war dieser brave Lehrer

Von dem Tobak ein Verehrer,

Was man ohne alle Frage

Nach des Tages Müh und Plage

Einem guten, alten Mann

Auch von Herzen gönnen kann. –

Max und Moritz, unverdrossen,

Sinnen aber schon auf Possen,

Ob vermittelst seiner Pfeifen

Dieser Mann nicht anzugreifen. –

– Einstens, als es Sonntag wieder

Und Herr Lämpel brav und bieder

In der Kirche mit Gefühle

Saß vor seinem Orgelspiele,

Schlichen sich die bösen Buben

In sein Haus und seine Stuben,

Wo die Meerschaumpfeife stan

## 4. RAG

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

retriever = db.as_retriever()

template = """Beantworte die Frage nur aufgrund der folgenenden Informationen:
{context}

Frage: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model_name = 'gpt-3.5-turbo'
model_name = 'gpt-4-1106-preview'
model = ChatOpenAI(model_name = model_name)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke(query)

'Der Lehrer heißt Herr Lehrer Lämpel.'